In [10]:
from abc import ABC, abstractmethod
import copy

In [182]:
class Board:
    def __init__(self):
        self.board = [[0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0]]
        
        self.blank = '  '
        self.winner = "none"
        self.captured_pieces_white = []
        self.captured_pieces_black = []
        self.history = []
        self.en_passant = False
    def __str__(self):
        disp_board = ""
        for i in range(64):
            
            if i%8 == 0:
                disp_board += str(8-int(i/8)) + " "
            
            this_space = self.board[int(i/8)][i%8]

            if this_space == 0:
                piece_to_add = " |"
            else:
                piece_to_add = str(this_space) + "|"
            disp_board += piece_to_add
            
            if i%8 == 7:
                disp_board = disp_board[:-1] + "\n"
        disp_board += "  a b c d e f g h"
        return disp_board

    def show_history(self, player_input):
        return self.history

    def split_notation(self, notation):
        notation_rank = list(notation)[1]
        notation_file = list(notation)[0]
        rank_to_y = {"a":0, "b":1, "c":2, "d":3, "e":4, "f":5, "g":6, "h":7}
        notation_x = 8 - int(notation_rank)
        notation_y = rank_to_y.get(notation_file)            
        return notation_x, notation_y

    def get_piece_at(self, position):

        if type(position) == str:
            position_xy = self.split_notation(position)
            i = position_xy[0]
            j = position_xy[1]
            return self.board[i][j]
        elif type(position) == tuple:
            i = position[0]
            j = position[1]
            return self.board[i][j]

    # checking move validity

    def is_piece(self, position):

        if self.get_piece_at(position) == 0:
            return False
        else:
            return True

    # uppercase is white
    
    def is_white(self, position):
        
        return self.get_piece_at(position).isupper()

    def get_valids(self, pos):
        piece_type = self.get_piece_at(pos)
        x = self.split_notation(pos)[0]
        y = self.split_notation(pos)[1]
        if self.is_white(pos):
            is_white = True
            piece_type = piece_type.lower()
        else:
            is_white = False
        
        if piece_type == "p":
            if is_white:
                capt = (self.is_piece((x-1, y-1)), self.is_piece((x+1, y-1)))
            else:
                capt = (self.is_piece((x-1, y+1)), self.is_piece((x+1, y+1)))
            en_p = (False, False)
            print(capt, x, y)
            pawn = Pawn(is_white, self.split_notation(pos), capt, en_p)
            return pawn.get_valids()
        elif piece_type == "r":
            rook = Rook(self.split_notation(pos))
            return rook.get_valids()
        elif piece_type == "n":
            knight = Knight(self.split_notation(pos))
            return knight.get_valids()
        elif piece_type == "b":
            bishop = Bishop(self.split_notation(pos))
            return bishop.get_valids()
        elif piece_type == "q":
            queen = Queen(self.split_notation(pos))
            return queen.get_valids()
        elif piece_type == "k":
            rooks_moved = (False , False)
            if is_white and pos == "e1":
                has_moved = False
                rooks_moved = (not self.get_piece_at("a1") == "R", not self.get_piece_at("h1") == "R")
            elif not is_white and pos == "e8":
                has_moved = False
                rooks_moved = (not self.get_piece_at("a8") == "r", not self.get_piece_at("h8") == "r")
            else:
                has_moved = True
                rooks_moved = (True, True)
            king = King(self.split_notation(pos), is_white, has_moved, rooks_moved)
            return king.get_valids()
        else:
            return("Error")

    def promote(self, pos, is_white):
        pass

    def display_valids(self, valid_list):

        # takes list of valid inputs created by get_valids() to generate visual. example:
        #    vals = chess.get_valids("e5")
        #    chess.display_valids(vals)
        
        valid_move_board = copy.deepcopy(self.board)
        
        for i in range(len(valid_list)):
            potential_x = valid_list[i][0]
            potential_y = valid_list[i][1]
            if valid_move_board[potential_x][potential_y] == 0:
                valid_move_board[potential_x][potential_y] = "x"
            else:
                pass
        
        disp_board = ""
        for i in range(64):
            
            if i%8 == 0:
                disp_board += str(8-int(i/8)) + " "
            
            this_space = valid_move_board[int(i/8)][i%8]

            if this_space == 0:
                piece_to_add = " |"
            else:
                piece_to_add = str(this_space) + "|"
            disp_board += piece_to_add
            
            if i%8 == 7:
                disp_board = disp_board[:-1] + "\n"
                
        disp_board += "  a b c d e f g h"

        print(disp_board)
        
    def move_is_valid(self, origin, destination):

        if self.is_piece(origin):

            
            
            if not self.is_piece(destination):
                return((True, "Piece moved"))

            elif self.is_white(origin) == self.is_white(destination):
                return((False, "Cannot capture your own piece"))

            else:
                return((True, "Piece captured"))
            
        else:
            return((False, "No piece there"))

    def make_move(self, origin, destination):
        origin_piece = self.get_piece_at(origin)
        origin_xy = self.split_notation(origin)
        origin_x = origin_xy[0]
        origin_y = origin_xy[1]

        
        dest_xy = self.split_notation(destination)
        dest_x = dest_xy[0]
        dest_y = dest_xy[1]

        move_validity = self.move_is_valid(origin, destination)

        if move_validity[0] and move_validity[1] == "Piece captured":
            dest_piece = self.get_piece_at(destination)
            dest_is_white = self.is_white(destination)
            if dest_is_white:
                self.captured_pieces_white.append(dest_piece)
            else:
                self.captured_pieces_black.append(dest_piece)
        if move_validity[0]:
            self.board[dest_x][dest_y] = self.board[origin_x][origin_y]
            self.history.append(f"{move_validity[1]}: {origin_piece} from {origin} to {destination}")
            self.board[origin_x][origin_y] = 0
            print(self)
            print(self.history[-1])
        else:
            print(move_validity[1])

In [192]:
class Pawn:
    def __init__(self, is_white, pos, capt, en_p):
        self.is_white = is_white
        self.pos = pos
        self.capt = capt
        self.en_p = en_p
        
    def get_valids(self):
        y = self.pos[0]
        x = self.pos[1]       
        if self.is_white:
            valid_moves = [(x, y-1)]
            if y == 6:
                valid_moves.append((x, y-2))
            if self.capt[0] or self.en_p[0]:
                valid_moves.append((x-1, y-1))
            if self.capt[1] or self.en_p[1]:
                valid_moves.append((x+1, y-1))
        else:
            valid_moves = [(x, y+1)]
            if y == 1:
                valid_moves.append((x, y+2))
            if self.capt[0] or self.en_p[0]:
                valid_moves.append((x-1, y+1))
            if self.capt[1] or self.en_p[1]:
                valid_moves.append((x+1, y+1))
        return valid_moves

In [193]:
class Knight:
    def __init__(self, pos):
        self.pos = pos
        self.has_moved = False 
    def get_valids(self):
        x = self.pos[0]
        y = self.pos[1]
        valid_moves = []
        moves = [(x-1, y-2), (x+1, y-2),  # up and left/right
                 (x-1, y+2), (x+1, y+2),  # down and left/right
                 (x+2, y-1), (x+2, y+1),  # right and up/down
                 (x-2, y-1), (x-2, y+1)]  # left and up/down

        for i in moves:
            if not 0 <= i[0] < 8 or not 0 <= i[1] < 8:
                pass
            else:
                valid_moves.append(i)

        return valid_moves

In [194]:
class Rook:
    def __init__(self, pos):
        self.pos = pos
        self.has_moved = False
        
    def get_valids(self):
        x = self.pos[0]
        y = self.pos[1]
        
        valid_moves_y = [(i, y) for i in range(8)] #  left or right
        valid_moves_y.remove((self.pos[0], self.pos[1]))
        
        valid_moves_x = [(x, i)for i in range(8)] # up or down              
        valid_moves_x.remove((self.pos[0], self.pos[1]))

        valid_moves = []

        for i in valid_moves_x:
            valid_moves.append(i)

        for i in valid_moves_y:
            valid_moves.append(i)
        
        return valid_moves

In [196]:
chess = Board()
chess.board[3][4] = "p"
chess.board[5][5] = "P"
print(chess)
a = chess.get_valids("f3")
print(a)

8  | | | | | | | 
7  | | | | | | | 
6  | | | | | | | 
5  | | | |p| | | 
4  | | | | | | | 
3  | | | | |P| | 
2  | | | | | | | 
1  | | | | | | | 
  a b c d e f g h
(False, False) 5 5
[(5, 4)]
